# Detecting anomalies on satellite images

This notebook contains an algorithm for comparing images with historical data and training an AI model suited for computer vision to recognize whether the differences present areas of interest, which can then be further investigated

### 1. Image request

#### Imports

In [1]:
from mundilib import MundiCatalogue

# other tools
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from detector import AnomalyDetector
from filter import ConvolutionalModel
from random import randint
from time import sleep

#### System mockup

In [ ]:
class Drone:
    def __init__(self, name="drone"):
        self.name = name

    def go(self, location) -> bool:
        print(f"Flying {self.name} to {location}")
        sleep(1)
        if randint(0,1) == 1:
            print("Anomaly confirmed, alerting personnel")
            return True
        else:
            print("False alarm, returning to base")
            return False

In [ ]:
drones = [Drone(f"drone {i}") for i in range(10)]

#### Prepare model

In [ ]:
detector = AnomalyDetector()
neural_net = ConvolutionalModel()

In [ ]:
def save_historical(img):
    detector.add_to_history(img)

def satellite_register_image(img):

    # find coordinates of possible areas of interest
    bounds = detector.detect_anomalies(img)

    for area in bounds:
        area_zoom = img[area[0]:area[2], area[1]:area[3]]
        # analyze in depth
        prediction = neural_net.predict(area_zoom)
        if prediction.item() == 1:
            # send drone to investigate
            drone =  drones[randint(0,len(drones)-1)]
            confirmed = drone.go(area)
            if not confirmed:
                neural_net.reveal_classification(area_zoom, 0)



#### Choice of satellite

In [2]:
c = MundiCatalogue()
wms = c.get_collection("Sentinel1").mundi_wms('GRD') # choice of satellite

#### Define WMS parameters

In [3]:
_projection_ = 'EPSG:4326'
_bbox_       = (32.495087,29.877812,32.558258,29.928541) # bbox Suez canal
_time_       = '2003-03-26'

_height_     = 600
_width_      = 1200

#### Fetch reference images of the area

In [ ]:
layers = list(wms.contents)

print (wms[layers[3]].title)

# getting image from 'wms'
img1 = wms.getmap(layers = [wms[layers[3]].name],
                         srs = _projection_,
                         bbox = _bbox_,
                         size = (_width_, _height_),
                         format ='image/png',
                         time = _time_,
                         showlogo = False,
                         transparent=False)

#display image
img_past = Image.open(img1)
save_historical(img_past)

_time_       = '2013-03-26'
img2 = wms.getmap(layers = [wms[layers[3]].name],
                         srs = _projection_,
                         bbox = _bbox_,
                         size = (_width_, _height_),
                         format ='image/png',
                         time = _time_,
                         showlogo = False,
                         transparent=False)

# display image
img_past = Image.open(img2)
save_historical(img_past)



#### Process incoming image

In [ ]:
current_time       = '2023-03-26'
img_current = wms.getmap(layers = [wms[layers[3]].name],
                         srs = _projection_,
                         bbox = _bbox_,
                         size = (_width_, _height_),
                         format ='image/png',
                         time = current_time,
                         showlogo = False,
                         transparent=False)

img_past = Image.open(img_current)
satellite_register_image(img_past)